In [174]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as signal
import skimage.filters as filters

from PIL import Image
from pyneurotrace import filters  as pntfilters
from scipy import integrate

<H3> Global Variables

In [175]:
# The Folder containing sub-directories to include in analysis
DATA_DIRECTORY = "osfstorage-archive/Data (Images)/"
# Frequency frames were collected
HZ = 10

In [176]:
"""
Jia, H., Rochefort, N. L., Chen, X., & Konnerth, A. (2011).
In vivo two-photon imaging of sensory-evoked dendritic calcium signals in cortical neurons.
Nature protocols, 6(1), 28.
"""
"""
Podgorski, K., & Haas, K. (2013).
Fast non‐negative temporal deconvolution for laser scanning microscopy.
Journal of biophotonics, 6(2), 153-162.
"""
"""
Coleman, P. (2019).
pyNeuroTrace. https://github.com/padster/pyNeuroTrace.git
"""
# To install pyneuortrace use this:
# pip install --upgrade "git+https://github.com/padster/pyNeuroTrace#egg=pyneurotrace&subdirectory=pyneurotrace"

'\nColeman, P. (2019).\npyNeuroTrace. https://github.com/padster/pyNeuroTrace.git\n'

In [177]:
# Change output figure size
# ...needs to be in its own cell for some reason...

plt.rcParams['figure.figsize'] = [20, 5]

In [178]:
# Take a folder of Tifs and turn it into a numpy array
def folder2tif(dir_path):
    final = []
    files = os.listdir(dir_path)
    files = sorted(files)
    movie = []
    for fname in files:
        im = Image.open(os.path.join(dir_path, fname))
        imarray = np.array(im)
        movie.append(imarray)
    movie = np.asarray(movie)
    return movie

# Returns change in fluorescence over average fluorescence of ROI
def deltaF(video_mask):
    video_mask_nan = video_mask.copy()
    video_mask_nan[video_mask_nan==0] = np.nan
    mean = np.mean(np.nanmean(video_mask))
    print(mean)
    dff= np.zeros((video_mask.shape[0]))
    for i in range(dff.shape[0]):
        delta = np.nanmean(video_mask[i, :, :])-mean
        dff[i] = delta/mean
    return dff

# Returns raw values fluorescence in the ROI
def rawIntensity(video_mask):
    video_mask_nan = video_mask.copy()
    video_mask_nan[video_mask_nan==0] = np.nan
    mean = np.nanmean(video_mask, axis=(1,2))
    return mean

# Generates the ROI 
def genROI(gcamp, rcamp):

    # To create a ROI for the nucleus a STD projection is created
    # Thresholding this image creates a mask for the roi
    std_projectionG = np.std(gcamp, axis=0)
    threshold = filters.threshold_otsu(std_projectionG)
    std_projectionG[std_projectionG < threshold] = 0
    std_projectionG[std_projectionG>0]=1

    # Create a ROI for the cytosl using an STD projection 
    # Thresholding this image creates a mask for the roi
    std_projectionR = np.std(rcamp, axis=0)
    threshold = filters.threshold_otsu(std_projectionR)
    std_projectionR[std_projectionR < threshold] = 0
    std_projectionR[std_projectionR>0]=1
    
    # Remove the Nuclear Mask from this ROI
    std_projectionR[std_projectionG==1]=0
    
    # Applying the masks for the two channels
    gcamp_masked = gcamp * std_projectionG
    rcamp_masked = rcamp * std_projectionR
    
    return gcamp_masked, rcamp_masked


# Generate a ROI for a cell
# Input of data directory, cell, Returns RCaMP and GCamP Raw Inrensity
def Cell2Trace(path, cell):
    # Import the movies provide the path to the folder containing the frames
    gcamp = folder2tif(path+cell+"_G/")
    rcamp = folder2tif(path+cell+"_R/")
    
    # Return Masked Arrays
    gcamp_masked, rcamp_masked = genROI(gcamp, rcamp)
    
    # Return Raw Traces from ROI
    gcamp = rawIntensity(gcamp_masked)
    rcamp = rawIntensity(rcamp_masked)
    
    return gcamp, rcamp

def peakDetect(trace):
    # Detect Peaks
    threshold =np.std(trace)
    peaks, _ = signal.find_peaks(trace, width=7, rel_height=.5, prominence=(.1*threshold))

    width = signal.peak_widths(trace, peaks, rel_height=.1)
    
    return peaks, width[0]

def signal_analysis(cell_id, gcamp, rcamp):
    gcamp_peaks, gcamp_widths = peakDetect(gcamp)
    rcamp_peaks, rcamp_widths = peakDetect(rcamp)

    
    # Match Peaks and puttin them in a list of tuples (g, r)
    gcamp_matched= []
    for g in gcamp_peaks:
        for r in rcamp_peaks:
            a = math.isclose(g, r, abs_tol=15)
            if a == True:
                gcamp_matched.append((g,r))
                
    # Assumption: Last shared peak is due to drug application
    # ***Improve this later***
    drug_app = np.mean(gcamp_matched[-1])
    cutoffG = np.where(gcamp_peaks == gcamp_matched[-1][0])[0][0]
    cutoffR = np.where(rcamp_peaks == gcamp_matched[-1][1])[0][0]
    gcamp_matched = gcamp_matched[:-1]


    # Apply the cutoff 
    gcamp_peaks = gcamp_peaks[:cutoffG]
    gcamp_widths = gcamp_widths[:cutoffG]
    rcamp_peaks = rcamp_peaks[:cutoffR]
    rcamp_widths = rcamp_widths[:cutoffR]
    
    
    # General Stats for the cell
    cell_stats = { 
                                'Cell ID': cell_id,
                                'GCaMP Peaks':len(gcamp_peaks),
                                'RCaMP Peaks':len(rcamp_peaks),
                                'Shared Peaks':len(gcamp_matched),
                                'GCaMP Percent Shared':(len(gcamp_matched)/len(gcamp_peaks)),
                                'RCaMP Percent Shared':(len(gcamp_matched)/len(gcamp_peaks)),
                                'Experiment Length (s)': gcamp.shape[0]/10,
                                'Predicted Drug Application': drug_app/10,
                               }
    
    cell_stats = pd.DataFrame(data=cell_stats, index=[0])
    peak_data = pd.DataFrame()
    for event in gcamp_matched:
        
        gindex = np.where(gcamp_peaks == event[0])[0]
        rindex = np.where(rcamp_peaks == event[1])[0]
        
        
        # Integrate Under the Curve for Area 
        # Note: Area from start to peak
        g_event_start = int(event[0]-gcamp_widths[gindex])
        if g_event_start < 0: 
            g_event_start = 0
        r_event_start = int(event[0]-rcamp_widths[rindex])
        if r_event_start < 0:
            r_event_start = 0

        g_area = integrate.cumtrapz(gcamp[g_event_start:event[0]])
        r_area = integrate.cumtrapz(rcamp[r_event_start:event[1]])
        
        peak_stats = {          'Cell ID': cell_id,            
                                'GCaMP Loc':event[0],
                                'GCaMP Width':gcamp_widths[gindex],
                                'GCaMP Prominence':gcamp[event[0]],
                                'GCaMP Area':g_area,                                
                                'RCaMP Loc':event[1],
                                'RCaMP Width':rcamp_widths[rindex],
                                'RCaMP Prominence':rcamp[event[1]],
                                'RCaMP Area':r_area,                                
                                'Promicence Ratio (G/R)':(gcamp[event[1]]/rcamp[event[0]]),
                                'Peak Time Diff (G-r)':((event[0]-event[1])*100),
                                                                 }
        peak_data = peak_data.append(peak_stats, ignore_index=True)
    return cell_stats, peak_data
    
    
    

In [179]:
# Collects all the cells in the analysis data directory and groups
# Them by condition in two lists
path = os.fspath(DATA_DIRECTORY)
cells = sorted((os.listdir(path)))
WT_Cells = []
YAC128_Cells = []
for folder in cells:
    print(folder)
    if 'WT'  in folder:
        WT_Cells.append(folder[:-2])
    if 'YAC128'  in folder:
        YAC128_Cells.append(folder[:-2])
        
WT_Cells = np.unique(WT_Cells)
YAC128_Cells = np.unique(YAC128_Cells)


Cell 14_YAC128_G
Cell 14_YAC128_R
Cell 17_WT_G
Cell 17_WT_R
Cell 18_WT_G
Cell 18_WT_R
Cell 22_YAC128_G
Cell 22_YAC128_R
Cell 29_YAC128_G
Cell 29_YAC128_R


# <h> Cycle Through WT Cells to Extract Peak Data

In [182]:
WT_Stats = pd.DataFrame()
WT_Peaks = pd.DataFrame()

for cell in WT_Cells:
    gcamp, rcamp = Cell2Trace(path, cell)
 
    # Calculate df/f and perform NND
    dffG = pntfilters.deltaFOverF0(gcamp, HZ)
    dffG = pntfilters.nndSmooth(dffG, HZ, tau=1)
    dffR = pntfilters.deltaFOverF0(rcamp, HZ)
    dffR = pntfilters.nndSmooth(dffR, HZ, tau=1)
    
    cell_stats, peak_data = signal_analysis(cell, dffG, dffR)
    
    WT_Stats = WT_Stats.append(cell_stats, ignore_index=True)
    WT_Peaks = WT_Peaks.append(peak_data, ignore_index=True)



# <h2> Display Results for WT

In [183]:
display(WT_Stats)
display(WT_Peaks)


,Cell ID,GCaMP Peaks,RCaMP Peaks,Shared Peaks,GCaMP Percent Shared,RCaMP Percent Shared,Experiment Length (s),Predicted Drug Application
0,Cell 17_WT,21,22,20,0.952381,0.952381,257.2,174.85
1,Cell 18_WT,8,18,8,1.000000,1.000000,252.9,187.95


,Cell ID,GCaMP Area,GCaMP Loc,GCaMP Prominence,GCaMP Width,Peak Time Diff (G-r),Promicence Ratio (G/R),RCaMP Area,RCaMP Loc,RCaMP Prominence,RCaMP Width
0,Cell 17_WT,"[0.23492248828641085, 0.5219932711418664, 0.85...",114.0,0.500777,[14.868113625566707],100.0,1.675790,"[0.22133598270784782, 0.4555607421482179, 0.70...",113.0,0.301117,[12.016280412917226]
1,Cell 17_WT,"[0.3916570767171559, 0.8787762332926485, 1.454...",226.0,0.925305,[14.71821469406919],-300.0,2.931590,"[0.2330621934128521, 0.47999020057628494, 0.73...",229.0,0.321727,[12.405036662951346]
2,Cell 17_WT,"[0.05011237464976387, 0.17949775093754877, 0.4...",327.0,0.888553,[15.230374299780692],0.0,3.726077,"[0.11181646486250597, 0.25424580125799434, 0.4...",327.0,0.238469,[14.92144630958785]
3,Cell 17_WT,"[0.4957083426453096, 1.0555738357761826, 1.669...",427.0,0.850693,[11.42226759529143],500.0,5.256002,"[0.15048967008068548, 0.30447014146585677, 0.4...",422.0,0.166401,[9.863722686788378]
4,Cell 17_WT,"[0.10685259646883505, 0.24371084220987746, 0.4...",487.0,0.213380,[6.9808726476127845],500.0,3.084750,[],482.0,0.062836,[5.135147612801404]
5,Cell 17_WT,"[0.06934006443532269, 0.15033606117950965, 0.2...",531.0,0.098373,[4.122019996593735],300.0,3.236305,[0.03565870511898457],528.0,0.035897,[4.46599366125929]
6,Cell 17_WT,"[1.7358236156279974, 3.6604192052493825, 5.742...",628.0,2.685433,[11.271025070602605],400.0,10.658677,"[0.20767226374636674, 0.4238926057903488, 0.64...",624.0,0.247598,[12.51313807892791]
7,Cell 17_WT,"[0.12520053204705642, 0.29526465497394405, 0.5...",720.0,0.356550,[9.410549600035665],800.0,3.846753,[],712.0,0.064776,[6.8910879127186035]
8,Cell 17_WT,"[0.3666512914227867, 0.933203071413891, 1.7239...",796.0,2.026235,[13.796371621650565],400.0,9.296942,"[0.1071957689626278, 0.23708778059382912, 0.38...",792.0,0.210882,[13.324160804806752]
9,Cell 17_WT,"[0.09942310836601106, 0.25287963726524565, 0.4...",892.0,0.393930,[10.263519044392638],600.0,6.096185,[],886.0,0.064551,[6.378130412098585]


# <h> Cycle Through YAC128 Cells to Extract Peak Data

In [185]:
YAC128_Stats = pd.DataFrame()
YAC128_Peaks = pd.DataFrame()

for cell in YAC128_Cells:
    gcamp, rcamp = Cell2Trace(path, cell)
 
    # Calculate df/f and perform NND
    dffG = pntfilters.deltaFOverF0(gcamp, HZ)
    dffG = pntfilters.nndSmooth(dffG, HZ, tau=1)
    dffR = pntfilters.deltaFOverF0(rcamp, HZ)
    dffR = pntfilters.nndSmooth(dffR, HZ, tau=1)
    
    cell_stats, peak_data = signal_analysis(cell, dffG, dffR)
    
    YAC128_Stats = YAC128_Stats.append(cell_stats, ignore_index=True)
    YAC128_Peaks = YAC128_Peaks.append(peak_data, ignore_index=True)



# <h2> Display Results for YAC128

In [186]:
display(YAC128_Stats)
display(YAC128_Peaks)


,Cell ID,GCaMP Peaks,RCaMP Peaks,Shared Peaks,GCaMP Percent Shared,RCaMP Percent Shared,Experiment Length (s),Predicted Drug Application
0,Cell 14_YAC128,8,10,8,1.0,1.0,343.2,272.25
1,Cell 22_YAC128,10,12,10,1.0,1.0,201.9,173.45
2,Cell 29_YAC128,8,12,8,1.0,1.0,243.9,183.40


,Cell ID,GCaMP Area,GCaMP Loc,GCaMP Prominence,GCaMP Width,Peak Time Diff (G-r),Promicence Ratio (G/R),RCaMP Area,RCaMP Loc,RCaMP Prominence,RCaMP Width
0,Cell 14_YAC128,"[0.15347080844950733, 0.3318137330628656, 0.53...",184.0,0.256778,[6.750700210647551],500.0,0.483330,[0.47760403382238614],179.0,0.484382,[6.7235056704006695]
1,Cell 14_YAC128,"[0.6986886889370509, 1.5194177981082555, 2.455...",427.0,1.303250,[8.967483465208375],800.0,1.649033,[],419.0,0.635422,[7.996602111470111]
2,Cell 14_YAC128,"[2.326393304392775, 4.830143808524468, 7.51968...",815.0,3.363962,[8.835688084254912],300.0,4.747125,[0.8550100408690486],812.0,0.871154,[4.967726818277924]
3,Cell 14_YAC128,"[0.12587412333938647, 0.37112219910573974, 0.8...",1554.0,3.667970,[14.237593571031539],-500.0,4.065624,"[0.5380006970414718, 1.1342531864894738, 1.781...",1559.0,0.926519,[10.575293625313634]
4,Cell 14_YAC128,"[0.054530094353410004, 0.17480163390335565, 0....",1792.0,3.595802,[14.401416571783784],-200.0,4.543565,"[0.27217062151923216, 0.6224568694084163, 1.04...",1794.0,0.794280,[12.617475156318278]
5,Cell 14_YAC128,"[0.025604059181182793, 0.09511532362750537, 0....",2179.0,3.481715,[14.429081394472178],-300.0,5.146848,"[0.19944103307608368, 0.4738495297938051, 0.81...",2182.0,0.683671,[12.613231022172386]
6,Cell 14_YAC128,"[0.11043951586898493, 0.3549729371431334, 0.81...",2364.0,3.247811,[14.910201110895741],-600.0,5.019510,"[0.3254523621300261, 0.7052040849606256, 1.130...",2370.0,0.669260,[11.741472135028289]
7,Cell 14_YAC128,"[0.10790600621086793, 0.2366152671943293, 0.38...",2503.0,0.200576,[8.315725864629258],600.0,2.385940,[],2497.0,0.083294,[6.101329464845094]
8,Cell 22_YAC128,"[0.49590326938563256, 1.0517206981435478, 1.66...",143.0,0.833636,[10.399505614405854],800.0,1.915302,"[0.3556775363493229, 0.7207191592174733, 1.092...",135.0,0.375308,[11.972554882073098]
9,Cell 22_YAC128,"[0.7033858941140222, 1.4819522656927149, 2.331...",272.0,1.156311,[10.501794980648924],700.0,2.839719,"[0.35378293231792796, 0.7169273887075507, 1.08...",265.0,0.380102,[12.209538513141695]
